In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [71]:
from numpy.random import RandomState

In [49]:
np.random.seed(42)

In [73]:
RandomState(5).permutation(14)

array([ 5,  1,  7,  2, 10, 13, 11,  4,  8,  9,  0, 12,  6,  3])

In [3]:
CV_Match = pd.read_excel("./data/kaggle/data.xlsx", sheet_name= "Match", index_col = 'Candidate_match') 
CV_Sourcing = pd.read_excel("./data/kaggle/data.xlsx", sheet_name = "Candidate", index_col = 'CV_ID')

In [32]:
CV_Sourcing

,Experience,Language Level - French
CV_ID,,
1000,NaN,NaN
1001,NaN,NaN
1002,NaN,NaN
1003,NaN,NaN
1004,NaN,NaN
...,...,...
995,NaN,NaN
996,NaN,NaN
997,NaN,NaN


Adding in the data augmentation steps as outlined here: https://docs.google.com/document/d/13W0YIM3UcS--UGs5K5s5_DHYr0cfRZC_QYr146-3AfI/edit#heading=h.4rkzkuexnsde

In [88]:
CV_Sourcing.Experience.notnull().sum()

786

In [7]:
CV_Sourcing['Language Level - French'].unique()

array([nan, 'B1', 'A1', 'C2', 'Native', 'B2', 'A2', 'C1'], dtype=object)

In [37]:
dfMerged = CV_Sourcing.merge(CV_Match, left_on=CV_Sourcing.index, right_on=CV_Match.index).rename(columns={'key_0':'CV_ID'}).sort_index()

In [44]:
statusMap = {
'A01 - Offer Preparation Started':1,
 'A03 - Offer accepted':1,
 'A03 - Offer declined':1,
 'A03 - Process Cancelled':1,
 'A04 - Hiring Request Started':1,
 'A04b – Hiring Started / Contract signed':1,
 'A05 - Onboarding':1,
 'Approved':0,
 'CV approved':0,
 'CV dropped out':0,
 'CV refused':0,
 'CV sent to France':0,
 'Candidate dropped out':0,
 'Candidate refused':0,
 'Candidate validated':1,
 'D01 – Resignation':1,
 'Dropped out':0,
 'Matched':0,
 'Rejected':0,
 'Sent to Client':0,
 'T01- Terminated':1
}

In [45]:
dfMerged['MatchStatus'] = dfMerged['Status'].map(statusMap).astype(int)

In [47]:
dfMerged.MatchStatus.value_counts()

0    1908
1     347
Name: MatchStatus, dtype: int64

In [48]:
from sklearn.model_selection import train_test_split

In [230]:
dfMerged.iloc[:,-1]

0       1
1       0
2       0
3       0
4       1
       ..
2250    0
2251    0
2252    0
2253    1
2254    0
Name: MatchStatus, Length: 2255, dtype: int32

In [76]:
Xtrain, Xtest, ytrain, ytest = train_test_split(dfMerged.iloc[:,:-1], dfMerged.iloc[:,-1], random_state=42, 
                                                test_size=.5, stratify=dfMerged.iloc[:,-1])

In [79]:
Xtrain.join(ytrain)['MatchStatus'].value_counts()

0    954
1    173
Name: MatchStatus, dtype: int64

In [80]:
Xtest.join(ytest)['MatchStatus'].value_counts()

0    954
1    174
Name: MatchStatus, dtype: int64

In [82]:
Xtest['Gender'] = 'M'
Xtrain['Gender'] = 'F'

In [96]:
baseGenderDF = pd.concat([Xtest.join(ytest), Xtrain.join(ytrain)]).sort_index()

In [97]:
baseGenderDF

,CV_ID,Experience,Language Level - French,Match_ID,Status,4. CV sent to Client,4. Client initial validation date,7. Client final approval date,Gender,MatchStatus
0,1025,[0-1],B1,10551,A03 - Offer declined,2017-03-15,2017-03-17,2017-04-05,M,1
1,1045,[0-1],B1,10234,Rejected,NaT,NaT,NaT,M,0
2,1045,[0-1],B1,10745,CV refused,2017-05-05,2017-05-05,NaT,M,0
3,1045,[0-1],B1,10840,CV refused,2017-05-23,2017-05-23,NaT,M,0
4,1045,[0-1],B1,10845,A03 - Offer declined,2017-05-23,2017-05-23,2017-06-30,M,1
...,...,...,...,...,...,...,...,...,...,...
2250,9821,NaN,NaN,13343,Sent to Client,2018-05-03,NaT,NaT,M,0
2251,9821,NaN,NaN,13437,CV approved,2018-05-04,2018-05-04,NaT,M,0
2252,9827,NaN,NaN,12367,CV refused,2018-02-27,2018-03-20,NaT,F,0
2253,9836,[0-1],NaN,12443,A05 - Onboarding,2017-12-01,2017-12-01,2017-12-01,F,1


In [231]:
baseGenderDF.groupby(['MatchStatus','Gender']).count()

CV_ID  Experience  Language Level - French  Match_ID  \
MatchStatus Gender                                                         
0           F         954         205                      172       954   
            M         954         205                      158       954   
1           F         173         169                      137       173   
            M         174         171                      133       174   

                    Status  4. CV sent to Client  \
MatchStatus Gender                                 
0           F          954                   629   
            M          954                   650   
1           F          173                   173   
            M          174                   174   

                    4. Client initial validation date  \
MatchStatus Gender                                      
0           F                                     659   
            M                                     670   
1           F                                     173   
            M                                     174   

                    7. Client final approval date  
MatchStatus Gender                                 
0           F                                 265  
            M                                 279  
1           F                                 173  
            M                                 174

In [105]:
int(len(baseGenderDF[(baseGenderDF.MatchStatus == 1)]) * (.5 + .02))

180

In [161]:
np.random.seed(42)
indM = np.random.choice(baseGenderDF[(baseGenderDF.MatchStatus == 1)].index, 180, replace=False)

In [162]:
len(indM)

180

In [163]:
len(baseGenderDF[(baseGenderDF.MatchStatus == 1)]) - 180

167

In [164]:
x = baseGenderDF[(baseGenderDF.MatchStatus == 1)]

In [172]:
baseGenderDF[~(baseGenderDF.MatchStatus == 1)]

,CV_ID,Experience,Language Level - French,Match_ID,Status,4. CV sent to Client,4. Client initial validation date,7. Client final approval date,Gender,MatchStatus
1,1045,[0-1],B1,10234,Rejected,NaT,NaT,NaT,M,0
2,1045,[0-1],B1,10745,CV refused,2017-05-05,2017-05-05,NaT,M,0
3,1045,[0-1],B1,10840,CV refused,2017-05-23,2017-05-23,NaT,M,0
5,1045,[0-1],B1,11124,Rejected,NaT,NaT,NaT,F,0
6,1060,NaN,NaN,10208,Rejected,NaT,NaT,NaT,M,0
...,...,...,...,...,...,...,...,...,...,...
2249,9821,NaN,NaN,12961,CV refused,2018-04-09,2018-04-09,NaT,F,0
2250,9821,NaN,NaN,13343,Sent to Client,2018-05-03,NaT,NaT,M,0
2251,9821,NaN,NaN,13437,CV approved,2018-05-04,2018-05-04,NaT,M,0
2252,9827,NaN,NaN,12367,CV refused,2018-02-27,2018-03-20,NaT,F,0


In [159]:
len(indM)

180

In [194]:
x[(x.index).isin(indM)].loc[:, "Gender"]

8       M
19      F
26      F
28      M
84      M
       ..
2108    F
2134    F
2193    F
2203    M
2214    F
Name: Gender, Length: 180, dtype: object

In [196]:
x[(x.index).isin(indM)]['Gender'] = 'M'

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [208]:
(x.index).isin(indM)

array([False, False, False,  True, False,  True,  True,  True, False,
        True,  True,  True, False, False, False,  True,  True,  True,
        True,  True, False, False,  True,  True,  True,  True, False,
       False, False, False,  True,  True, False,  True, False, False,
       False,  True,  True,  True, False, False,  True, False, False,
        True,  True, False, False, False, False, False, False, False,
       False,  True,  True,  True, False,  True,  True, False, False,
        True, False, False,  True,  True,  True,  True, False, False,
        True,  True, False,  True,  True,  True,  True,  True, False,
        True,  True, False,  True, False,  True, False, False, False,
        True, False,  True,  True,  True, False,  True,  True, False,
       False, False,  True, False, False,  True, False, False, False,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True, False, False,  True, False,  True,  True,
        True, False,

In [221]:
# indF = 
x[~x.index.isin(indM)].index

Int64Index([   0,    4,    7,   13,   50,  100,  116,  117,  131,  134,
            ...
            2009, 2039, 2071, 2078, 2092, 2119, 2145, 2186, 2234, 2253],
           dtype='int64', length=167)

In [216]:
x.loc[~indM, 'Gender'] 

KeyError: "None of [Int64Index([-2024,  -593, -2065, -1575, -1939,  -673, -1150, -1278, -1512,\n             -801,\n            ...\n            -1270,  -984,  -583, -1254, -1683,  -631, -2013, -2098, -1443,\n            -1984],\n           dtype='int64', length=180)] are in the [index]"

In [214]:
x.loc[indM, 'Gender'].unique()

array(['M'], dtype=object)

In [166]:
x[~(x.index).isin(indM)]

,CV_ID,Experience,Language Level - French,Match_ID,Status,4. CV sent to Client,4. Client initial validation date,7. Client final approval date,Gender,MatchStatus
0,1025,[0-1],B1,10551,A03 - Offer declined,2017-03-15,2017-03-17,2017-04-05,M,1
4,1045,[0-1],B1,10845,A03 - Offer declined,2017-05-23,2017-05-23,2017-06-30,M,1
7,1071,[0-1],A1,10567,A05 - Onboarding,2017-03-17,2017-03-17,2017-03-17,F,1
13,1128,[2-4],C2,12390,A05 - Onboarding,2017-05-12,2017-05-13,2017-05-14,M,1
50,1197,[8+],A2,12394,A05 - Onboarding,2017-03-14,2017-03-14,2017-03-14,F,1
...,...,...,...,...,...,...,...,...,...,...
2119,9154,[8+],NaN,12762,A04b – Hiring Started / Contract signed,2018-03-20,2018-03-21,2018-03-28,M,1
2145,9250,[8+],B1,12413,A05 - Onboarding,2017-08-05,2017-08-05,2017-08-05,M,1
2186,951,[8+],Native,11070,A05 - Onboarding,2017-07-18,2017-07-18,2017-08-08,F,1
2234,9800,[8+],NaN,12869,A04 - Hiring Request Started,2018-04-04,2018-04-10,2018-04-24,M,1


In [170]:
pd.concat(x

,CV_ID,Experience,Language Level - French,Match_ID,Status,4. CV sent to Client,4. Client initial validation date,7. Client final approval date,Gender,MatchStatus
0,1025,[0-1],B1,10551,A03 - Offer declined,2017-03-15,2017-03-17,2017-04-05,M,1
4,1045,[0-1],B1,10845,A03 - Offer declined,2017-05-23,2017-05-23,2017-06-30,M,1
7,1071,[0-1],A1,10567,A05 - Onboarding,2017-03-17,2017-03-17,2017-03-17,F,1
8,1081,[8+],NaN,11753,A04b – Hiring Started / Contract signed,2018-02-06,2018-02-23,2018-03-19,M,1
13,1128,[2-4],C2,12390,A05 - Onboarding,2017-05-12,2017-05-13,2017-05-14,M,1
...,...,...,...,...,...,...,...,...,...,...
2193,9586,NaN,NaN,11751,A05 - Onboarding,2018-02-06,2018-02-20,2018-02-28,F,1
2203,9675,[8+],A1,11775,A04b – Hiring Started / Contract signed,2018-02-12,2018-02-13,2018-02-15,M,1
2214,9703,[0-1],NaN,12487,A03 - Offer declined,2018-03-14,2018-03-14,2018-03-07,F,1
2234,9800,[8+],NaN,12869,A04 - Hiring Request Started,2018-04-04,2018-04-10,2018-04-24,M,1


In [137]:
x.index.difference(indM)

Int64Index([   0,    7,    8,   19,   26,   28,   84,   85,   94,  121,
            ...
            2090, 2092, 2097, 2104, 2105, 2119, 2145, 2186, 2193, 2253],
           dtype='int64', length=206)

In [224]:
# Deviations from demographic parity 
def deviation_func_gender(x, df, colName='Gender'):
    yesDF = df[(df.MatchStatus == 1)]
    noDF = df[~(df.MatchStatus == 1)]
    totalMalePercent = .5 + (x/100)
    totalMales = int(len(yesDF) * (totalMalePercent))
    
    np.random.seed(42)
    indM = np.random.choice(yesDF.index, totalMales, replace=False)
    indF = yesDF[~yesDF.index.isin(indM)].index
    
    yesDF.loc[indM, colName] = 'M'
    yesDF.loc[indF, colName] = 'F'
    
    dfNew = pd.concat([yesDF, noDF])
    return dfNew

In [238]:
deviation0 = deviation_func_gender(0, baseGenderDF)
deviation2 = deviation_func_gender(2, baseGenderDF)
deviation4 = deviation_func_gender(4, baseGenderDF)
deviation6 = deviation_func_gender(6, baseGenderDF)
deviation8 = deviation_func_gender(8, baseGenderDF)
deviation10 = deviation_func_gender(10, baseGenderDF)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [ ]:
deviation0.to_csv('./data/kaggle/base-gender-dataset.csv')
deviation2.to_csv('./data/kaggle/base-gender-2-percent-dataset.csv')

In [239]:
deviation0.groupby(['MatchStatus','Gender']).count()

CV_ID  Experience  Language Level - French  Match_ID  \
MatchStatus Gender                                                         
0           F         954         205                      172       954   
            M         954         205                      158       954   
1           F         174         172                      135       174   
            M         173         168                      135       173   

                    Status  4. CV sent to Client  \
MatchStatus Gender                                 
0           F          954                   629   
            M          954                   650   
1           F          174                   174   
            M          173                   173   

                    4. Client initial validation date  \
MatchStatus Gender                                      
0           F                                     659   
            M                                     670   
1           F                                     174   
            M                                     173   

                    7. Client final approval date  
MatchStatus Gender                                 
0           F                                 265  
            M                                 279  
1           F                                 174  
            M                                 173

In [240]:
deviation2.groupby(['MatchStatus','Gender']).count()

CV_ID  Experience  Language Level - French  Match_ID  \
MatchStatus Gender                                                         
0           F         954         205                      172       954   
            M         954         205                      158       954   
1           F         167         165                      129       167   
            M         180         175                      141       180   

                    Status  4. CV sent to Client  \
MatchStatus Gender                                 
0           F          954                   629   
            M          954                   650   
1           F          167                   167   
            M          180                   180   

                    4. Client initial validation date  \
MatchStatus Gender                                      
0           F                                     659   
            M                                     670   
1           F                                     167   
            M                                     180   

                    7. Client final approval date  
MatchStatus Gender                                 
0           F                                 265  
            M                                 279  
1           F                                 167  
            M                                 180

In [229]:
test

,CV_ID,Experience,Language Level - French,Match_ID,Status,4. CV sent to Client,4. Client initial validation date,7. Client final approval date,Gender,MatchStatus
0,1025,[0-1],B1,10551,A03 - Offer declined,2017-03-15,2017-03-17,2017-04-05,F,1
4,1045,[0-1],B1,10845,A03 - Offer declined,2017-05-23,2017-05-23,2017-06-30,F,1
7,1071,[0-1],A1,10567,A05 - Onboarding,2017-03-17,2017-03-17,2017-03-17,M,1
8,1081,[8+],NaN,11753,A04b – Hiring Started / Contract signed,2018-02-06,2018-02-23,2018-03-19,M,1
13,1128,[2-4],C2,12390,A05 - Onboarding,2017-05-12,2017-05-13,2017-05-14,F,1
...,...,...,...,...,...,...,...,...,...,...
2249,9821,NaN,NaN,12961,CV refused,2018-04-09,2018-04-09,NaT,F,0
2250,9821,NaN,NaN,13343,Sent to Client,2018-05-03,NaT,NaT,M,0
2251,9821,NaN,NaN,13437,CV approved,2018-05-04,2018-05-04,NaT,M,0
2252,9827,NaN,NaN,12367,CV refused,2018-02-27,2018-03-20,NaT,F,0


In [31]:
sorted(CV_Match.Status.unique().tolist())

['A01 - Offer Preparation Started',
 'A03 - Offer accepted',
 'A03 - Offer declined',
 'A03 - Process Cancelled',
 'A04 - Hiring Request Started',
 'A04b – Hiring Started / Contract signed',
 'A05 - Onboarding',
 'Approved',
 'CV approved',
 'CV dropped out',
 'CV refused',
 'CV sent to France',
 'Candidate dropped out',
 'Candidate refused',
 'Candidate validated',
 'D01 – Resignation',
 'Dropped out',
 'Matched',
 'Rejected',
 'Sent to Client',
 'T01- Terminated']

In [30]:
CV_Match[CV_Match.Status == 'A04b – Hiring Started / Contract signed']

,Match_ID,Status,4. CV sent to Client,4. Client initial validation date,7. Client final approval date
Candidate_match,,,,,
8855,11626,A04b – Hiring Started / Contract signed,2018-01-04,2018-01-08,2018-01-17
9207,11689,A04b – Hiring Started / Contract signed,2018-01-22,2018-03-22,2018-03-28
1081,11753,A04b – Hiring Started / Contract signed,2018-02-06,2018-02-23,2018-03-19
9675,11775,A04b – Hiring Started / Contract signed,2018-02-12,2018-02-13,2018-02-15
2518,12486,A04b – Hiring Started / Contract signed,2018-03-14,2018-03-14,2018-03-05
7186,12575,A04b – Hiring Started / Contract signed,2018-03-06,2018-03-12,2018-03-21
9154,12762,A04b – Hiring Started / Contract signed,2018-03-20,2018-03-21,2018-03-28
12521,12860,A04b – Hiring Started / Contract signed,2018-03-29,2018-03-29,2018-03-26


In [20]:
CV_Match[CV_Match.index == 356].sort_values(by='Match_ID')

,Match_ID,Status,4. CV sent to Client,4. Client initial validation date,7. Client final approval date
Candidate_match,,,,,
356,517,Matched,NaT,NaT,NaT
356,10963,Rejected,NaT,2018-03-30,2018-03-30
356,11252,Rejected,2018-03-27,2018-03-27,2017-09-18
356,11268,Candidate dropped out,2017-09-25,2017-09-25,2018-04-03


In [18]:
x = CV_Match.groupby(CV_Match.index).count()
x[x.Match_ID > 1]

,Match_ID,Status,4. CV sent to Client,4. Client initial validation date,7. Client final approval date
Candidate_match,,,,,
349,4,4,2,3,1
352,6,6,3,4,1
353,2,2,1,1,0
355,2,2,1,1,0
356,4,4,2,3,3
...,...,...,...,...,...
13126,3,3,1,1,1
13172,2,2,2,2,0
13213,2,2,1,0,0


In [11]:
CV_Match[CV_Match['7. Client final approval date'].notnull()]

,Match_ID,Status,4. CV sent to Client,4. Client initial validation date,7. Client final approval date
Candidate_match,,,,,
830,10002,Candidate refused,2017-01-03,2017-01-03,2017-02-02
822,10005,Candidate refused,2017-01-05,2017-01-05,2017-02-09
1745,10007,Rejected,NaT,NaT,2018-03-30
1747,10008,A03 - Offer declined,2017-01-23,2017-01-24,2017-04-11
815,10010,D01 – Resignation,2016-12-20,2016-12-20,2016-12-22
...,...,...,...,...,...
552,9984,A03 - Offer declined,2016-11-23,2016-11-23,2016-12-01
583,9985,A05 - Onboarding,2016-11-28,2016-11-29,2016-12-06
584,9986,Rejected,NaT,2018-03-30,2018-03-30


In [ ]:
# joining the two tables
CV_Sourcing.merge()

In [110]:
CV_Sourcing['Gender'] = np.random.choice(['M','F'], 10099, p=[0.65, 0.35])

In [112]:
CV_Sourcing['Race'] = np.random.choice(['American Indian or Alaska Native','Asian',
                                       'Black or African American','Native Hawaiian or Other Pacific Islander',
                                        'White', 'NA'], 10099, p=[0.05, 0.3, .18, .07, .35, .05])

In [116]:
CV_Sourcing['Parent'] = np.random.choice([1,0], 10099, p=[0.30, 0.7])

In [129]:
CV_Sourcing['City'] = np.random.choice(['Atlanta, GA','San Francisco, CA',
                                       'Chicago, IL','New York City, NY',
                                        'Houston, TX', 'Dallas, TX', 
                                        'Salt Lake City, UT', 'Miami, FL',
                                       'Los Angeles, CA','Seattle, WA'], 10099, 
                                       p=[0.07, 0.18, .09, .11, .07, .1, .04, .12, .1, .12])

In [131]:
CV_Sourcing['Occupation'] = np.random.choice(['Software Developer','Administrative Assistant',
                                       'Customer Service','Cook',
                                        'High-skilled Sales', 'Low-skilled Sales'], 10099, 
                                       p=[0.43, 0.12, .2, .12, .1, .03])

In [133]:
CV_Sourcing

,Experience,Language Level - French,Gender,Race,Parent,City,Occupation
CV_ID,,,,,,,
1000,NaN,NaN,F,NA,0,"Seattle, WA",Software Developer
1001,NaN,NaN,F,White,0,"Seattle, WA",Customer Service
1002,NaN,NaN,M,Asian,1,"Chicago, IL",Customer Service
1003,NaN,NaN,M,White,1,"Dallas, TX",Software Developer
1004,NaN,NaN,F,Asian,1,"Salt Lake City, UT",Software Developer
...,...,...,...,...,...,...,...
995,NaN,NaN,F,Asian,0,"New York City, NY",High-skilled Sales
996,NaN,NaN,F,Native Hawaiian or Other Pacific Islander,1,"Seattle, WA",Cook
997,NaN,NaN,M,Black or African American,0,"San Francisco, CA",Software Developer


In [134]:
CV_Sourcing.to_excel('./data/kaggle/augmented_data.xlsx')